In [1]:
import os
import sys
from pathlib import Path
# Setting the working directory to the root of the project
project_dir = Path("C:/Users/adbou/source/repos/KFHXRelatedAi/")
os.chdir(project_dir)

from Configs.GeneralPaths import SOURCEDATA
import pandas as pd
import numpy as np
import ast


In [2]:
#load user transactions data
deals_embeddings = pd.read_csv(Path(SOURCEDATA / "Deals_Embeddings.csv"))
deals_embeddings['ada_embedding'] = deals_embeddings['ada_embedding'].apply(ast.literal_eval)



In [3]:
deals_embeddings.head()

,ContentId,ada_embedding
0,113815,"[-0.013089276850223541, 0.026867099106311798, ..."
1,113816,"[-0.05805347114801407, -0.007304157596081495, ..."
2,113817,"[-0.04382414370775223, 0.0026055192574858665, ..."
3,113818,"[-0.01307600922882557, 0.009115846827626228, -..."
4,113819,"[-0.02055640146136284, -0.014343508519232273, ..."


In [4]:
#loading user transactions
user_transactions = pd.read_excel(Path(SOURCEDATA / "Transaction_User.xlsx"))
user_tranactions = user_transactions.drop(columns=['TrxId'])
user_tranactions.head()

,FK_BusinessUserId,PointsRedeemed,FK_ContentId
0,976480,1000,113923
1,976480,10000,113853
2,976480,10000,113853
3,976921,10,113851
4,976480,50,113835


In [5]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_similarity(embedding, embeddings):
    similarities = cosine_similarity([embedding], embeddings)
    return similarities[0]


In [6]:
def recommend_deals_for_user(user_id, user_deals_df, deals_embeddings_new):
    # Get the user's redeemed deals' content IDs
    redeemed_content_ids = user_deals_df[user_deals_df['FK_BusinessUserId'] == user_id]['FK_ContentId'].unique()
    
    # Retrieve the embeddings for the redeemed deals
    redeemed_embeddings = deals_embeddings_new[deals_embeddings_new['ContentId'].isin(redeemed_content_ids)]
    
    if redeemed_embeddings.empty:
        print("No redeemed deals found for this user.")
        return pd.DataFrame()
    
    # Get all embeddings
    all_embeddings = deals_embeddings_new['ada_embedding'].tolist()

    # Initialize an empty list to store similarities
    similarities = []

    # Calculate similarities for each redeemed embedding
    for embedding in redeemed_embeddings['ada_embedding']:
        similarity = calculate_cosine_similarity(embedding, all_embeddings)
        similarities.append(similarity)

    # Average the similarities to get a single similarity score for each deal
    average_similarities = np.mean(similarities, axis=0)

    # Add the similarities to the DataFrame
    deals_embeddings_new['similarity'] = average_similarities

    # Filter out deals the user has already redeemed
    recommended_deals = deals_embeddings_new[~deals_embeddings_new['ContentId'].isin(redeemed_content_ids)]

    # Sort by similarity in descending order and take the top 5 recommendations
    top_5_recommendations = recommended_deals.sort_values(by='similarity', ascending=False).head(5)

    return top_5_recommendations




In [7]:
user_id = 1064756

# Get recommendations for the user
top_5_recommendations = recommend_deals_for_user(user_id, user_tranactions, deals_embeddings)

In [8]:
top_5_recommendations

,ContentId,ada_embedding,similarity
14,113829,"[-0.037020616233348846, 0.03241224214434624, 0...",0.617364
102,113917,"[-0.025992970913648605, 0.00470952782779932, -...",0.613336
140,115095,"[-0.020191777497529984, -0.03286787122488022, ...",0.594669
71,113886,"[-0.02998787723481655, -0.03537878766655922, -...",0.561173
62,113877,"[-0.03304778411984444, 0.023215385153889656, -...",0.511509


In [9]:
user_trx = user_tranactions[user_tranactions['FK_BusinessUserId'] == user_id]
user_trx

,FK_BusinessUserId,PointsRedeemed,FK_ContentId
37103,1064756,100,115094
